In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)



In [2]:

def centroid(cluster):
    return np.mean(cluster, axis=0)

def sse(data, labels):
    res = 0
    N = data.shape[0]  # Numero di punti dati nel dataset
    centroid_total = np.mean(data, axis=0)  # Calcolo del centroide totale
    total_variance = np.sum(np.square(data - centroid_total))  # Calcolo della varianza totale dei dati
    
    for cluster in np.unique(labels):
        current_cluster = data[labels == cluster]
        cluster_centroid = centroid(current_cluster)
        cluster_variance = np.sum(np.square(current_cluster - cluster_centroid))  # Varianza dei dati nel cluster
        res += cluster_variance * current_cluster.shape[0]  # Somma delle distanze quadrate nel cluster

    sse_normalized = res /  total_variance  # Normalizzazione combinata dell'SSE
    return sse_normalized

In [ ]:
#import dataset
dataset =  fetch_openml("pendigits")
X = dataset.data
y = dataset.target

print(X.shape)
print(X.head)

print(y.head)


## Preprocessing

### Dataset Analysis

In [ ]:
#original data box plot
plt.figure(figsize=(20, 10))
sns.boxplot(X)
plt.title("Box plot delle features")
plt.xlabel("Features")
plt.show()


In [ ]:
#check missing values
print(X.isnull().sum())

In [ ]:
#check outliers
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1
print(((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).sum())
print(X.shape)

#outliers removal
Y = y[~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)]
X = X[~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)]

#data box plot
plt.figure(figsize=(20, 10))
sns.boxplot(X)
plt.title("Box plot delle features")
plt.xlabel("Features")
plt.show()
print(((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).sum())
print(X.shape)

#plot data distribution after the removal of outliers
x_array = np.array(X)
plt.figure(figsize=(20, 10))
sns.scatterplot(x = x_array[:, 0], y = x_array[:, 1],hue = Y)
plt.xlabel('input1')
plt.ylabel('input2')
plt.title('Distribuzione dati originali')
plt.show()


### data trasformazione

In [ ]:
#data standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

plt.figure(figsize=(20, 10))
sns.boxplot(X_scaled)

plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_scaled[:, 0], y = X_scaled[:, 1],hue = Y)
plt.xlabel('input1')
plt.ylabel('input2')
plt.title('distribuzione dati standardizzati')
plt.show()

In [ ]:
#data normalization
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
X_normalized = normalizer.fit_transform(X)
plt.figure(figsize=(20, 10))
sns.boxplot(X_normalized)

plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_normalized[:, 0], y = X_normalized[:, 1],hue = Y)
plt.xlabel('input1')
plt.ylabel('inptu2')
plt.title('distribuzione dati normalizzati')
plt.show()

### Dimensionality reduction settings

In [9]:
#dataset to be used for dimensionality reduction:
#   X -> original dataset 
#   X_scaled -> standardized Dataset 
#   X_normalized -> normalized dataset
X_or = X
X = X_scaled

### PCA ### 

In [ ]:
#determine the number of principal components using the elbow rule
pca = PCA()
pca.fit(X)
plt.figure(figsize=(20, 10))
#plt.plot(pca.explained_variance_ratio_, marker='o',)
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o', )
plt.grid()
plt.xticks(np.arange(0, 16, 1))
plt.legend(['cumulative explained variance'])


plt.xlabel('number of components')
plt.ylabel('explained variance')
plt.title("Elbow method per PCA")
plt.show()

pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)



In [ ]:
#plot of 2D data of the first 2 principal components
plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_pca[:, 0], y = X_pca[:, 1],hue = Y)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('PCA')
plt.show()

### LDA

In [ ]:
#determine the number of principal components using the elbow rule
lda = LinearDiscriminantAnalysis()
lda.fit(X, Y)

plt.figure(figsize=(20, 10))
plt.plot(np.cumsum(lda.explained_variance_ratio_), marker='o')
plt.grid()
plt.xticks(np.arange(0, 16, 1))
plt.xlabel('number of components')
plt.ylabel('explained variance')
plt.title("Elbow method per LDA")
plt.show()

lda = LinearDiscriminantAnalysis(n_components=4)
X_lda = lda.fit_transform(X, Y)
print(X_lda.shape)





In [ ]:

#plot of the data in 2D
plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_lda[:, 0], y = X_lda[:, 1],hue = Y)
plt.xlabel('LDA 1')
plt.ylabel('LDA 2')
plt.title('LDA')
plt.show()


### TSNE

In [ ]:
#t-SNE
tsne = TSNE(n_components=2, random_state=0)
X_tsne = tsne.fit_transform(X)
print(X_tsne.shape)

In [ ]:
#plot of the data in 2D
plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_tsne[:, 0], y = X_tsne[:, 1],hue = Y)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('t-SNE')
plt.show()

## Clustering

### Settings clustering

In [17]:

#dataset da usare per il clustering:
#   X -> original Dataset 
#   X_pca -> Dataset after PCA
#   X_tsne -> Dataset after t-SNE
#   X_lda -> Dataset after LDA
X_test = X_tsne
tsne = TSNE(n_components=2, random_state=0)
X_tsne_or = tsne.fit_transform(X_or)
X_test_or = X_tsne_or

### K-Means

In [ ]:
#k-means
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0)
kmeans.fit(X_test)

labels = kmeans.labels_

#plot of the data in 2D
plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_test[:, 0], y = X_test[:, 1],hue = labels, palette='pastel')
#centroids plot
for i in range(10):
    cen = (centroid(X_test[labels == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('K-means')
plt.show()

print("SSE: ", sse(X_test, labels))

In [ ]:
#kmeans original datas
kmeans_or = KMeans(n_clusters=10, random_state=0)
kmeans_or.fit(X_test_or)
labels_or = kmeans_or.labels_

#plot of the origianl datas in 2D
plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_test_or[:, 0], y = X_test_or[:, 1],hue = labels_or, palette='pastel')
#centroids plot
for i in range(10):
    cen = (centroid(X_test_or[labels_or == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('K-means dati originali')
plt.show()

print("SSE dati originali: ", sse(X_test_or, labels_or))

### Hierarchical Agglomerative Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage


hierarchical_clustering = AgglomerativeClustering(n_clusters=10, metric='euclidean', linkage='ward')
hierarchical_clustering.fit(X_test)

labels = hierarchical_clustering.labels_
linked = linkage(X_test, 'ward')

#plot of the data in 2D
plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_test[:, 0], y = X_test[:, 1],hue = labels, palette='pastel')
#centroids plot
for i in range(10):
    cen = (centroid(X_test[labels == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Hierarchical Agglomerative Clustering')
plt.show()

plt.figure(figsize=(20, 10))
dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=False)
plt.title('Dendrogram for Hierarchical Clustering')
plt.show()

print("SSE: " + str(sse(X_test, labels)))

In [ ]:
#hac origianl datas
hierarchical_clustering_or = AgglomerativeClustering(n_clusters=11, metric='euclidean', linkage='ward')
hierarchical_clustering_or.fit(X_test_or)

labels_or = hierarchical_clustering_or.labels_
linked_or = linkage(X_test_or, 'ward')

#plot dei dati originali in 2D
plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_test_or[:, 0], y = X_test_or[:, 1],hue = labels_or, palette='pastel')
#centroids plot
for i in range(10):
    cen = (centroid(X_test_or[labels_or == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')

plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Hierarchical Agglomerative Clustering con dati originali')
plt.show()

#dendogram origianl data
'''plt.figure(figsize=(20, 10))
dendrogram(linked_or, orientation='top', distance_sort='descending', show_leaf_counts=False)
plt.title('Dendrogram for Hierarchical Clustering con dati originali')
plt.show()'''

print("SSE dati originali: " + str(sse(X_test_or, labels_or)))

In [ ]:
#find optimal number of clusters
sse_hca = []

for N in range(2,16):
    hierarchical_clustering = AgglomerativeClustering(n_clusters=N, metric='euclidean', linkage='ward')
    hierarchical_clustering.fit(X_test)

    labels = hierarchical_clustering.labels_
    linked = linkage(X_test, 'ward')
    sse_hca.append(sse(X_test,labels))



In [ ]:
plt.figure()
plt.plot(sse_hca, marker='o')
plt.title("SSE of HAC clustering")
plt.xlabel("number of clusters")
plt.ylabel("SSE value")
plt.xticks(np.arange(0, 16, 1))
plt.grid()


### DBSCAN

#### DBSCAN with default parameteres

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
dbscan.fit(X_test)

labels = dbscan.labels_

plt.figure(figsize=(20, 10))
p = sns.scatterplot(x = X_test[:, 0], y = X_test[:, 1],hue = labels, palette='tab10')
plt.xlabel('x')
plt.ylabel('y')
plt.title('DBSCAN')
p.get_legend().remove()
for i in range(len(np.unique(labels))):
    cen = (centroid(X_test[labels == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')

plt.show()

print("SSE: " + str(sse(X_test, labels)))
print("Numero di cluster: " + str(len(np.unique(labels))))

In [ ]:
#dbscan with origianl datas
dbscan_or = DBSCAN()
dbscan_or.fit(X_test_or)

labels_or = dbscan_or.labels_

plt.figure(figsize=(20, 10))
p = sns.scatterplot(x = X_test_or[:, 0], y = X_test_or[:, 1],hue = labels_or, palette='tab10')
plt.xlabel('x')
plt.ylabel('y')
plt.title('DBSCAN dati originali')
p.get_legend().remove()
for i in range(len(np.unique(labels_or))):
    cen = (centroid(X_test_or[labels_or == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')

plt.show()

print("SSE dati originali: " + str(sse(X_test_or, labels_or)))
print("Numero di cluster dati originali: " + str(len(np.unique(labels_or))))

#### DBSCAN with personalized parameters

In [ ]:
dbscan = DBSCAN(eps=8.2, min_samples=33, metric='euclidean')
dbscan.fit(X_test)
labels = dbscan.labels_

plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_test[:, 0], y = X_test[:, 1],hue = labels, palette='pastel')
for i in range(len(np.unique(labels))):
    cen = (centroid(X_test[labels == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.title('DBSCAN')
plt.show()

print("SSE: " + str(sse(X_test, labels)))

In [ ]:
dbscan = DBSCAN(eps=9.5, min_samples=33, metric='euclidean')
dbscan.fit(X_test)
labels = dbscan.labels_

plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_test[:, 0], y = X_test[:, 1],hue = labels, palette='pastel')
for i in range(len(np.unique(labels))):
    cen = (centroid(X_test[labels == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.title('DBSCAN')
plt.show()

print("SSE: " + str(sse(X_test, labels)))
print("Numero di cluster: " + str(len(np.unique(labels))))

In [ ]:
#dbscan with origianl datas
dbscan_or = DBSCAN(eps=8.2, min_samples=33, metric='euclidean')
dbscan_or.fit(X_test_or)
labels_or = dbscan_or.labels_


plt.figure(figsize=(20, 10))
sns.scatterplot(x = X_test_or[:, 0], y = X_test_or[:, 1],hue = labels_or, palette='pastel')
for i in range(len(np.unique(labels_or))):
    cen = (centroid(X_test_or[labels_or == i]))
    plt.text(cen[0], cen[1],"X", fontsize=20, color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.title('DBSCAN dati originali')
plt.show()

print("SSE: " + str(sse(X_test_or, labels_or)))
print("Numero di cluster: " + str(len(np.unique(labels_or))))